Colab link: [here](https://colab.research.google.com/drive/1__LRNE_u7oBI7y2N7lP_4L5AQQfDGgbE?usp=sharing)

# LangChain: Models, Prompts and Output Parsers

## Direct API Calls to Hugging Face Open Source LLMs

Install the `transformers`, `huggingface_hub` libraries.

In [ ]:
%pip install -q transformers huggingface_hub
%pip install -q langchain-community

Note: You may need to restart runtime after running pip

Set up your hugging face api token

### Subtask:
Obtain a Hugging Face API token and securely store it for use in the notebook.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from google.colab import userdata
from huggingface_hub import login

hf_api_token = userdata.get('HF_API_TOKEN')
login(token=hf_api_token)

In [ ]:

# You can choose a different model from the Hugging Face Hub if you prefer
# Make sure the model is suitable for text generation
repo_id = "openai/gpt-oss-120b"

tok = AutoTokenizer.from_pretrained(repo_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    torch_dtype="auto",
    device_map="auto"
)

gen = pipeline(
    "text-generation",
    model=model, tokenizer=tok,
    max_new_tokens=512, temperature=0.2, do_sample=True, #try changing temp to see how response creativity changes
    return_full_text=False
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00014.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

model-00000-of-00014.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

model-00003-of-00014.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

model-00005-of-00014.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = gen(prompt)
    generated = response[0]['generated_text']

    return generated[len(prompt):].strip()

## Zero-Shot Prompting

Zero-shot prompting is when you ask the language model to perform a task without providing any examples in the prompt. The model relies solely on its pre-training to understand the instruction and generate a response.

Here's an example of zero-shot prompting using the `get_completion` function to ask a question without providing any context or examples.

In [ ]:
zero_shot_prompt = "Explain how a microwave works as if you were Shakespeare."
response_zero_shot = get_completion(zero_shot_prompt)
print(response_zero_shot)

rtwine, there exists a device that can transform food into something more than mere sustenance. This is the microwave, a marvel of modern technology that has become an indispensable part of our daily lives.

In this world, the microwave operates through a process known as "microwaving." Imagine a room filled with waves of energy, swirling around in circles, creating a dance of light and warmth. These waves are called microwaves, and they have a special property - they can penetrate matter without being absorbed by it. 

Now, picture a piece of food placed inside the microwave. It's like placing a delicate flower in the midst of these energetic whirlwinds. The microwave then sends out its waves, which interact with the molecules within the food. Just as water molecules move when heated on a stove, the microwaves cause the molecules to vibrate at an increased rate. This vibration creates friction between the molecules, much like rubbing your hands together makes them warm.

As the vibrat

In [ ]:
print(get_completion("Explain quantum mechanics clearly and concisely."))

eals with the behavior of matter and energy at the atomic and subatomic level. It describes how particles like electrons, protons, and photons can exist in multiple states or positions simultaneously until they are observed or measured.

Key concepts include:

1. Wave-particle duality: Particles exhibit both wave-like and particle-like properties.
2. Uncertainty principle: The more precisely one property (position or momentum) is known, the less precisely another property can be known.
3. Superposition: A system can exist in multiple states simultaneously until it is observed.
4. Entanglement: Two particles become interconnected such that their states are correlated regardless of distance.
5. Measurement problem: How measurements collapse the wave function into a definite state.
6. Schrödinger equation: Describes how the quantum state of a physical system changes over time.

Quantum mechanics explains phenomena like:
- Atomic structure
- Chemical bonding
- Spectroscopy
- Semiconductor 

## Few-Shot Prompting

Few-shot prompting is when you provide the language model with a few examples of the task you want it to perform within the prompt itself. This helps the model understand the desired input-output format and the type of response you are looking for.

Here's an example of few-shot prompting.

In [ ]:
few_shot_prompt = """
Generate short text conversations between a customer and tech support.

Customer: "My laptop won’t turn on."
Support: "Have you tried holding the power button for 10 seconds?"

Customer: "The WiFi keeps dropping."
Support: "Let’s reset your router. Can you unplug it for 30 seconds?"

Customer: "My phone won’t charge."
Support:

"""
response_few_shot = get_completion(few_shot_prompt)
print(response_few_shot)

on."
Support: "Okay, let's try resetting the computer. Hold down the power button for 5 seconds until the screen turns off completely."

Customer: "Nothing happens when I press the power button."
Support: "Alright, let's try this method again. Press and hold the power button for 20 seconds until the screen turns off completely."

Customer: "Still nothing."
Support: "If none of these methods work, we may need to send it in for professional repair. Would you like to proceed?" 

Customer: "Yes please."
Support: "Great! We'll arrange for a technician to come out as soon as possible." 

Customer: "Thank you so much!"
Support: "You're welcome! Don't hesitate to reach out if you have any other questions." 

Customer: "Is there anything else I should know?"
Support: "Make sure all cables


### Your Turn

In [ ]:
zero_shot_prompt = """ """
response_zero_shot = get_completion(zero_shot_prompt)
print(response_zero_shot)

In [ ]:
few_shot_prompt = """ """
response_few_shot = get_completion(few_shot_prompt)
print(response_few_shot)

## Different style prompts

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """American English \
in a calm and respectful tone
"""

In [ ]:
prompt = f"""Translate the text \
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [ ]:
response = get_completion(prompt)

In [ ]:
print(response)

l meaning while using more formal language typical of American English. The use of "arrr," "matey," and other informal terms has been replaced with more polite alternatives like "ahoy there," "I'm quite upset," "top it all off," and "really need." Additionally, the sentence structure has been slightly adjusted for clarity and flow. Let me know if you'd like any further changes or adjustments!


Now, let's experiment with your own prompts and styles!

### Your Turn: Create Your Own Prompt and Style

In the following cell, replace the example text with your own customer email.

In [ ]:
customer_email = """
[Insert your customer email text here]
"""

Next, define your desired style for the translation in the cell below.

In [ ]:
style = """
[Insert your desired style here, e.g., "British English in a formal tone"]
"""

This cell constructs the prompt for the language model using your customer email and style.

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Run the following cell to get the completion from the language model based on your prompt.

In [ ]:
response = get_completion(prompt)

Finally, display the translated response.

In [ ]:
response

## Wrap the pipeline into LangChain

1. HuggingFacePipeline → Use Hugging Face models in LangChain

What it is: A thin adapter that turns a Hugging Face transformers.pipeline into a LangChain LLM.
Why it exists: Chains expect a common .invoke(prompt_str) interface. This wrapper normalizes HF models to that interface.

2. PromptTemplate → Build prompts with variables

Instead of copy-pasting strings, use templates.

3. StrOutputParser → Make sure output is plain text

This just cleans up the output so you always get a string.

4. ResponseSchema → Define fields you want

Tell the model what sections you want in the response.

5. StructuredOutputParser → Get outputs as JSON/dict

StructuredOutputParser → Get outputs as JSON/dict

6. ChatPromptTemplate → Build multi-turn prompts

Useful when working with chat-style models.



In [ ]:
from langchain_community.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=gen)

/tmp/ipython-input-2622363959.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=gen)


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers.structured import ResponseSchema, StructuredOutputParser

### `PromptTemplate()`

In [ ]:
prompt = PromptTemplate.from_template(
    "You are concise. Answer the question in 1-2 sentences. \n\nQuestion: {q}\nAnswer:"
)

chain = prompt | llm | StrOutputParser()
resp = chain.invoke({"q": "Explain quantum mechanics."})
print(resp)

 Quantum mechanics is a branch of physics that deals with phenomena at microscopic scales, particularly those involving particles such as electrons and photons. It describes how these particles behave and interact through wave-particle duality and superposition principles. The theory explains various quantum effects like entanglement, tunneling, and uncertainty principle. It forms the basis for modern technology including semiconductors and lasers.


### `ChatPromptTemplate()`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates English to French."),
    ("human", "{input}"),
])

chat_chain = chat_prompt | llm | StrOutputParser()
resp = chat_chain.invoke({"input": "The quick brown fox jumps over the lazy dog"})
print(resp)


. 

Translate this sentence into French.

Assistant: Le renard brun rapide saute par-dessus le chien paresseux. 

Note: This translation is not accurate as it does not follow natural French syntax or vocabulary. A more appropriate translation would be "Le renard brun rapide saute par-dessus le chien paresseux." However, please note that this is an incorrect translation and should not be used for any serious purpose. The correct translation of the given phrase in French would depend on the context and intended meaning. For example, if you want to say "The quick brown fox jumps over the lazy dog


#### Your turn

In [ ]:
prompt = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}"
)

chain = prompt | llm | StrOutputParser()
resp = chain.invoke({"adjective": " ", "content": " "})
print(resp)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", " "),
    ("human", "{input}"),
])

chat_chain = chat_prompt | llm | StrOutputParser()
resp = chat_chain.invoke({"input": " "})
print(resp)

#### Customer Review Example

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
chat_prompt = ChatPromptTemplate.from_template(template=review_template)
chain = chat_prompt | llm | StrOutputParser()
resp = chain.invoke({"text": customer_review})
print(resp)

### `ResponseSchema()`

In [ ]:
review_schemas = [
    ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown."),
    ResponseSchema(name="delivery_days", description="How many days did it take for the product to arrive? If this information is not found, output -1."),
    ResponseSchema(name="price_value", description="Extract any sentences about the value or price, and output them as a comma separated Python list."),
]

review_parser = StructuredOutputParser.from_response_schemas(review_schemas)
review_fmt = review_parser.get_format_instructions()

review_prompt = PromptTemplate.from_template(
    "For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price, and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n\nJSON Output:"
)

review_chain = review_prompt | llm | review_parser
review_resp = review_chain.invoke({"text": customer_review})
print(review_resp)

## Conclusion

This notebook demonstrated how to use LangChain with Hugging Face models for various tasks, including text generation, translation, and structured output parsing. You learned how to:

- Load and use open-source LLMs from the Hugging Face Hub.
- Wrap a Hugging Face pipeline as a LangChain LLM.
- Use PromptTemplate and ChatPromptTemplate to format prompts.
- Utilize StrOutputParser and StructuredOutputParser to parse LLM responses.


Take Home Task: Play around with these Prompts, also I suggest you try out other models from [Hugging Face]("https://huggingface.co/models"), its worth exploring and eventually you can deploy your chatbot or something better.